### Shared Word

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
lang_folder = "German"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language
lang_pair = "Portuguese"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language 

In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [5]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [6]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/German/German_And_Portuguese (por)_Etymologeek_Result_All.xlsx']

In [7]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,dict_entry_main,portuguese_word
0,rein,puro
1,pur,puro
2,teufel,diabo
3,kurz,baixo
4,typ,tipo
...,...,...
296,liberal,liberal
297,kanton,cantão
298,broccoli,brócolis
299,laktose,lactose


In [8]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique","bikini"]

In [9]:
words = df_pair["dict_entry_main"].values.tolist()

In [10]:
disable_set = set(disable_list)
words_set = set(words)

In [11]:
word_list = list(words_set.difference(disable_set))

In [12]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three", "Four", "Five", "Sentence"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}.xlsx", index=False)
    

#### Concat Result

In [13]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,sirene
1,dom
2,pfirsich
3,master
4,moses
...,...
293,coup
294,fair
295,lampe
296,cäsar


In [14]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,kurz rein,77
1,brief schreiben,58
2,sicherheit garantieren,43
3,kurz pause,40
4,runde golf,20
...,...,...
85,tschüss garten,3
86,service medaille,3
87,service typ,3
88,pizza pause,3


In [15]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram


In [16]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram


In [17]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram


In [18]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,meister fan,12
1,tschüss linda,7
2,whisky pur,6
3,stress pur,5
4,runde golf,4
5,tschüss meister,4
6,golf tennis,4
7,tschüss garten,3
8,sack master,3
9,literatur mode,2


In [19]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,sirene,kurz rein,77.0,NaN,NaN,NaN,NaN,NaN,NaN,meister fan,12.0
1,dom,brief schreiben,58.0,NaN,NaN,NaN,NaN,NaN,NaN,tschüss linda,7.0
2,pfirsich,sicherheit garantieren,43.0,NaN,NaN,NaN,NaN,NaN,NaN,whisky pur,6.0
3,master,kurz pause,40.0,NaN,NaN,NaN,NaN,NaN,NaN,stress pur,5.0
4,moses,runde golf,20.0,NaN,NaN,NaN,NaN,NaN,NaN,runde golf,4.0
...,...,...,...,...,...,...,...,...,...,...,...
293,coup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,fair,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,lampe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,cäsar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx", index=False)

In [21]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [22]:
df_word_order_twogram = word_in_twogram(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_twogram(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_twogram(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_twogram(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_twogram(df_shared_all, "word", "sentence") 

In [23]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()

In [24]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [25]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,alkohol,"alkohol rein, liter alkohol",NaN,NaN,NaN,bar alkohol
1,ball,ball rein,NaN,NaN,NaN,kevin ball
2,bande,"bande rein, orange bande",NaN,NaN,NaN,NaN
3,bar,"dollar bar, bar kurz, bar service, bar dollar",NaN,NaN,NaN,bar alkohol
4,baron,brief baron,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
93,mode,NaN,NaN,NaN,NaN,literatur mode
94,pudding,NaN,NaN,NaN,NaN,karamell pudding
95,rose,NaN,NaN,NaN,NaN,rose engel
96,stern,NaN,NaN,NaN,NaN,bronze stern


In [26]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency.xlsx", index=False)

#### Copy Move And Delete

In [27]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*.xlsx")
output_file

['German_Portuguese_Shared_Twogram.xlsx',
 'German_Portuguese_Shared_Threegram.xlsx',
 'German_Portuguese_Shared_Fourgram.xlsx',
 'German_Portuguese_Shared_Fivegram.xlsx',
 'German_Portuguese_Shared_Sentence.xlsx',
 'German_Portuguese_Shared_Result_With_Frequency.xlsx',
 'German_Portuguese_Shared_Join_Result_Without_Frequency.xlsx']

In [28]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [29]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass